In [1]:
from openai_retrievers import CustomMultiQueryRetriever, CustomContextualCompressionRetriever, VectorStoreRetriever
from prompting import ChainOfThoughtComposer
from config import openai_key
from functions import devrev_functions
import logging
import json

### Let's setup logging, to catch any errors that might creep up

In [2]:
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

## Setup Retriever

In [3]:
retriever = VectorStoreRetriever(
    openai_key=openai_key,
    name="cmqr_vs_0",
    init_functions=devrev_functions
)

INFO:faiss.loader:Loading faiss with AVX2 support.
INFO:faiss.loader:Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
INFO:faiss.loader:Loading faiss.
INFO:faiss.loader:Successfully loaded faiss.


## Setup Composer

In [4]:
composer = ChainOfThoughtComposer(
    openai_api_key=openai_key,
    temperature=0.0,
    max_tokens=1000,
)

## Run a Query

In [5]:
query = "List all high severity tickets coming in from slack from customer Cust123 and generate a summary of them."
functions = retriever.find_functions(query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [6]:
#format the answer json
answer = composer(query=query, functions=functions)
answer_json = json.loads(answer['text'])
answer_json

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[{'tool_name': 'search_object_by_name',
  'arguments': [{'argument_name': 'query', 'argument_value': 'Cust123'}]},
 {'tool_name': 'works_list',
  'arguments': [{'argument_name': 'ticket.rev_org',
    'argument_value': '$$PREV[0]'},
   {'argument_name': 'ticket.source_channel', 'argument_value': ['slack']},
   {'argument_name': 'ticket.severity', 'argument_value': ['high']}]},
 {'tool_name': 'summarize_objects',
  'arguments': [{'argument_name': 'objects', 'argument_value': '$$PREV[1]'}]}]